## **Mount Google Drive to store files & data**

In [49]:
gan_name = "art_generator"

## **1) Importing Python Packages for GAN**


In [50]:
import os
from os.path import join, exists
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import time
import random

In [51]:
# from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D, Dense, Conv2DTranspose
from keras.layers import Dropout
from keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam


In [52]:
##!unzip /content/drive/MyDrive/TensorFlow/datasets/paintings15k.zip -d /content/drive/MyDrive/TensorFlow/datasets/


In [53]:
dataset_path = "F:\\Real-time-style-transfers\\data"
#images_path = join(dataset_path, "bobross")
#images_path = join(dataset_path, "fine_art")
#images_path = join(dataset_path, "train_slim")
images_path = join(dataset_path, "paintings15k")
resized_path = join(dataset_path, "resized_images")
generated_path = join(dataset_path, "generated_images")

In [54]:
if not exists(resized_path):
  os.mkdir(resized_path)

In [55]:
if not exists(generated_path):
  os.mkdir(generated_path)

## **2) Parameters for Neural Networks & Data**

In [56]:
img_width = 256
img_height = 256
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)
batch_size = 10

In [57]:
seed = int(time.time())
def seed_everything(seed=seed):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything()

In [58]:
cv2.__version__

'4.6.0'

## **Resizing Data to match Neural Network Input**

In [59]:
def resize_images(inpath, outpath, size):
  print("Resizing Images")
  i = 0
  for image in os.listdir(inpath):
    if exists(f"{outpath}/{image}"):
        continue
    try:
        img = cv2.imread(join(inpath, image))
        img = cv2.resize(img, size)
        cv2.imwrite(f"{outpath}/{image}",img)
    except:
        print(image)
    # print(img.shape)
    i = i+1

In [60]:
# Decode a single image in a TFRecord
# It also standardizes the image
def decode_image(image) -> tf.Tensor:
    #img = (tf.cast(image, tf.float32) / 127.5) - 1
    img = image / 127.5 - 1
    print(type(img))
    img = tf.reshape(img, [img_width, img_height, channels])
    #print(img.shape)
    return img

In [61]:
def dataset_from_dir(dataset_folder, ordered=False):
    dataset = tf.keras.utils.image_dataset_from_directory(
        dataset_folder,
        labels=None,
        #validation_split=0.2,
        #subset="training",
        shuffle = False,
        seed=seed,
        image_size=(img_height, img_width),
        batch_size=1)
    dataset = dataset.map(decode_image)
    dataset = dataset.cache()
    #dataset.shuffle(2048)
    return dataset.batch(batch_size)

In [62]:
def dataset_manual(dataset_folder):
    array = []
    for image in os.listdir(resized_path):
        image = Image.open(join(resized_path, image))
        data = np.asarray(image)
        array.append(data)

    X_train = np.array(array)
    X_train = X_train / 127.5 -1.
    print(X_train.shape)
    return X_train

In [63]:
# Colab was running out of memory because the datasets were huge. This solves that. 
def load_batch(path, batchSize):
    array = []
    files = os.listdir(path) # I can further optimize by caching this to global
    total = len(files)
    indexes = np.random.randint(0, total, batchSize)
    for i in indexes:
        image = Image.open(join(path, files[i]))
        data = np.asarray(image)
        array.append(data)

    train = np.array(array)
    train = train / 127.5 -1.
    return train
    

In [ ]:
#load_batch(resized_path, batch_size)

In [65]:
reshape_size = (img_width,img_height)
resize_images(images_path, resized_path, reshape_size)

Resizing Images
b6114bbd4cbaf708300cdbe71b88d689c.jpg


In [66]:
#X_train = dataset_from_dir(resized_path)
#epoch_batches = len(X_train)

In [67]:
#X_train = dataset_manual(resized_path)
#epoch_batches = int(len(X_train) / batch_size)

In [68]:
epoch_batches = int(len(os.listdir(resized_path)) / batch_size)
epoch_batches

1487

## **3) Building Generator**





In [69]:
def build_generator():
    multiplyer = int(((img_width/2)/2/2))
    model = Sequential()
    model.add(Dense(256 * multiplyer * multiplyer, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((multiplyer,multiplyer,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
  
    model.summary()

    return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 262144)            26476544  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 262144)            0         
                                                                 
 reshape (Reshape)           (None, 32, 32, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 64, 64, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 128, 128, 128)    262272    
 ranspose)                                              

## **4) Building Discriminator**

In [70]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, (3,3), padding='same', ))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256, 256, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 128)     73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 256, 256, 128)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 128)     147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256, 256, 128)     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 256)    

## **5) Connecting Neural Networks to build GAN**

In [71]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)
GAN.summary()
GAN.compile(loss='binary_crossentropy', optimizer=adam)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       27528963  
                                                                 
 sequential_1 (Sequential)   (None, 1)                 17295617  
                                                                 
Total params: 44,824,580
Trainable params: 27,528,963
Non-trainable params: 17,295,617
_________________________________________________________________


In [72]:
# generator.summary()
# discriminator.summary()

## **6) Outputting Images**


In [73]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cv2.imwrite(f"{generated_path}/{epoch}_{gan_name}_{cnt}.jpg",(gen_imgs[cnt] * 0.5 + 0.5))
            #cv2.imwrite(f"{generated_path}/{epoch}_{gan_name}_{cnt}.jpg",(gen_imgs[cnt] * 255))
            cnt += 1
    print(f"Saving Image to {generated_path}")
    fig.savefig("currentgeneration.png")
    fig.savefig(f"{generated_path}/{epoch}_{gan_name}_plot.png")
    plt.close()

## **7) Training GAN**

In [ ]:
def train(epochs, batch_size=32):
    #(X_train, _), (_, _) = cifar10.load_data()

    bat_per_epo = epoch_batches
    print(bat_per_epo)
    # X_train = np.expand_dims(X_train, axis=3)
    # print(X_train.shape)

    #Create our Y for our Neural Networks
    valid = np.ones((batch_size, 1))
    fakes = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        for j in range(bat_per_epo):
            #Get Random Batch
            #idx = np.random.randint(0, epoch_batches, batch_size)
            imgs = load_batch(resized_path, batch_size)

            #Generate Fake Images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator.predict(noise)

            #Train discriminator
            d_loss_real = discriminator.train_on_batch(imgs, valid)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, latent_dim))

            # inverse y label
            g_loss = GAN.train_on_batch(noise, valid)

    print(f"******* E: {epoch} [D loss: {d_loss[0]}, acc: {100* d_loss[1]:.4f}] [G loss: {g_loss}]")
    # if(epoch % save_interval) == 0:
    save_imgs(epoch)


train(100, batch_size=10)

1487
1/1 [==============================] - ETA: 0s

In [ ]:
noise = np.random.normal(0, 1, (16, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs[2])

In [ ]:
plt.imshow(gen_imgs[6])

In [ ]:
generator.save_weights("/content/drive/MyDrive/models/generator1hour.h5")
discriminator.save_weights("/content/drive/MyDrive/models/discriminator1hour.h5")

### **8) Making GIF**

In [ ]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)